In [ ]:
!pip install contractions
import contractions
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import pickle

from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
# import xgboost as xgb
from sklearn.calibration import CalibratedClassifierCV
import re 
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_label = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
sub=pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [ ]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

Определим список стоп-слов, из из них удалим те стоп-слова, которые означают отрицание

In [ ]:
contractions.fix( "couldn't")
not_stop = set(['don', 'hadn','couldn','haven','wasn','doesn', 
            'hasn', 'aren', 'shouldn', 'mightn', 'don', 'needn',
            'weren','didn','wouldn','isn', 
           ])
stop_words = set(stopwords.words('english'))
stop_words  = stop_words.difference(not_stop)
# Раскрываем сокрощения типа haven't -> have not и убираем из стоп-слов
stop_words = [st for st in stop_words if 'not' not in contractions.fix(st)]


In [ ]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_features=10000)#10k

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
#     stop_words='english',
    ngram_range=(2, 5),#2-6
    max_features=50000)#50k
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

if True:
    with open('word_vectorizer.pk', 'wb') as f:
        pickle.dump(word_vectorizer, f)
    with open('char_vectorizer.pk', 'wb') as f:
        pickle.dump(char_vectorizer, f)

In [ ]:
if False:
    word_vectorizer = pickle.load(open("word_vectorizer.pk", "rb"))
    char_vectorizer = pickle.load(open("char_vectorizer.pk", "rb"))

    train_word_features = word_vectorizer.transform(train_text)
    test_word_features = word_vectorizer.transform(test_text)

    train_char_features = char_vectorizer.transform(train_text)
    test_char_features = char_vectorizer.transform(test_text)

    train_features = hstack([train_char_features, train_word_features])
    test_features = hstack([test_char_features, test_word_features])

In [ ]:
del train_word_features, test_word_features,train_char_features, test_char_features,
del word_vectorizer, char_vectorizer

In [ ]:
%%time
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag',n_jobs=-1)
#     svm = LinearSVC(dual=False)
#     classifier = AdaBoostClassifier(cv, algorithm='SAMME', n_estimators=100)
#     svm = LinearSVC()
#     classifier = CalibratedClassifierCV(svm) 
#     classifier = xgb.XGBClassifier(objective='multi:softprob', learning_rate=1,
#                                    tree_method='auto',
#                                    max_depth = 12,    
#                                    silent=True, 
#                                    n_estimators=100, 
#                                    num_class=2, )

#     print('class')
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    with open(f'logreg_{class_name}.pk', 'wb') as f:
        pickle.dump(classifier, f)

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('submission.csv', index=False)